# Wikipedia爬蟲練習

In [72]:
import requests
import re
from bs4 import BeautifulSoup
import urllib.parse as up
import os

In [103]:
key_word = "國立清華大學"  
utf8_keyword = up.quote(key_word)
print(f'{key_word} : {utf8_keyword}')
root_url = '/wiki/'+utf8_keyword
print(f'ROOT URL: {root_keyword_link}')

國立清華大學 : %E5%9C%8B%E7%AB%8B%E6%B8%85%E8%8F%AF%E5%A4%A7%E5%AD%B8
ROOT URL: /wiki/%E7%B6%B2%E8%B7%AF%E7%88%AC%E8%9F%B2


## 作業：接下來定義一個爬蟲函數，這個函數的主要工作為：
### (1) 爬取當前關鍵字的解釋，並存入檔案(因為文章內容太多會佔滿整個頁面，所以存程檔案，方便後續檢視)
### (2) 萃取出當前關鍵字所引用的外部連結，當作新的查詢關鍵字
### (3) 把第(2)擷取到的關鍵字當作新的關鍵字，回到第(1)步，爬取新的關鍵字解釋。

In [100]:
def WikiArticle(key_word, root_url, recursive, max_recursive_depth=1):
    
    #max recursive depth
    if (recursive <= max_recursive_depth):
        print(f"遞迴層[{recursive}] - {root_url} ({key_word})")
        
        # request/soup
        headers = {
            'authority': 'zh.wikipedia.org',
            'method': 'GET',
            'path': root_url,
            'scheme': 'https',
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
            'accept-encoding': 'gzip, deflate, br',
            'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
            'cookie': 'GeoIP=TW:TPE:Taipei:25.05:121.53:v4; TBLkisOn=0; mwPhp7Seed=8b8; WMF-Last-Access-Global=04-Jun-2019; WMF-Last-Access=04-Jun-2019',
            'dnt': '1',
            #'if-modified-since': 'Tue, 04 Jun 2019 12:03:22 GMT',
            'referer': 'https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'}   

        url = 'https://zh.wikipedia.org' + root_url  
        res = requests.get(url, headers=headers)
        res.encoding = 'utf-8'
        soup = BeautifulSoup(res.text, "lxml")
        content = soup.find('div', attrs={'id':'mw-content-text'}).find_all(name='p')
        
        # save file by key_word
        file_name = 'WikiArticle'
        with open(os.path.join(file_name,key_word+'.txt'),'w') as f:
            for paragraph in content:
                if file_name not in os.listdir():
                    os.mkdir(file_name)
                f.write(paragraph.text) 
  
        #save external link to dict
        external_link_dict = dict({})
        for ext_link in content:
            a_tag = ext_link.find_all('a', href=re.compile("^(/wiki/)([\w//%]+)$"))
            for link in a_tag:
                external_link_dict[link['href']] = link.text
                    
        #loop func
        if (len(external_link_dict) > 0): # check 
            recursive += 1 
            for k, v in external_link_dict.items():
                WikiArticle(k,v,recursive,max_recursive_depth) 
                

### 執行前個步驟定義好的爬蟲主程式

In [101]:
# 定義爬取的遞迴深度。深度不要訂太深，否則會爬很久。
max_recursive_depth = 1
WikiArticle('國立清華大學',root_url,0,max_recursive_depth) #太多了 直接interrupt

遞迴層[0] - /wiki/%E5%9C%8B%E7%AB%8B%E6%B8%85%E8%8F%AF%E5%A4%A7%E5%AD%B8 (國立清華大學)
遞迴層[1] - /wiki/%E5%9C%B0%E7%90%86%E5%9D%90%E6%A0%87 (座標)
遞迴層[1] - /wiki/%E5%93%88%E4%BD%9B%E7%87%95%E4%BA%AC%E5%AD%B8%E7%A4%BE (哈佛燕京學社)
遞迴層[1] - /wiki/%E7%92%B0%E5%A4%AA%E5%B9%B3%E6%B4%8B%E5%A4%A7%E5%AD%B8%E8%81%AF%E7%9B%9F (環太平洋大學聯盟)
遞迴層[1] - /wiki/%E5%9C%8B%E9%9A%9B%E5%95%86%E7%AE%A1%E5%AD%B8%E9%99%A2%E4%BF%83%E9%80%B2%E5%8D%94%E6%9C%83 (國際商管學院促進協會)
遞迴層[1] - /wiki/AACSB (AACSB)
遞迴層[1] - /wiki/%E9%AB%98%E6%95%99%E6%B7%B1%E8%80%95%E8%A8%88%E7%95%AB (高教深耕計畫)
遞迴層[1] - /wiki/%E5%8C%97%E4%BA%AC (北京)
遞迴層[1] - /wiki/%E5%A4%A7%E6%B8%85%E5%B8%9D%E5%9C%8B (大清帝國)
遞迴層[1] - /wiki/%E6%B8%B8%E7%BE%8E%E5%AD%A6%E5%8A%A1%E5%A4%84 (遊美學務處)
遞迴層[1] - /wiki/%E5%8F%B0%E7%81%A3 (臺灣)
遞迴層[1] - /wiki/%E6%96%B0%E7%AB%B9%E5%B8%82 (新竹市)
遞迴層[1] - /wiki/%E5%9C%8B%E7%AB%8B%E4%BA%A4%E9%80%9A%E5%A4%A7%E5%AD%B8 (國立交通大學)
遞迴層[1] - /wiki/%E6%96%B0%E7%AB%B9%E7%A7%91%E5%AD%B8%E5%B7%A5%E6%A5%AD%E5%9C%92%E5%8D%80 (新竹科學工業園區)
遞迴層[1] - /wiki/%E6%A2%85%E

遞迴層[1] - /wiki/%E5%BC%A0%E5%AD%90%E9%AB%98 (張子高)
遞迴層[1] - /wiki/%E4%B8%AD%E5%8E%9F%E5%A4%A7%E6%88%B0 (中原大戰)
遞迴層[1] - /wiki/%E9%96%BB%E9%8C%AB%E5%B1%B1 (閻錫山)
遞迴層[1] - /wiki/%E5%96%AC%E8%90%AC%E9%81%B8 (喬萬選)
遞迴層[1] - /wiki/%E5%86%9B%E9%98%80 (軍閥)
遞迴層[1] - /wiki/%E6%95%99%E8%82%B2%E9%83%A8 (教育部)
遞迴層[1] - /wiki/%E5%90%B4%E5%8D%97%E8%BD%A9 (吳南軒)
遞迴層[1] - /wiki/%E8%B5%84%E9%87%91 (資金)
遞迴層[1] - /wiki/%E6%95%99%E6%8E%88 (教授)
遞迴層[1] - /wiki/%E6%9C%B1%E8%87%AA%E6%B8%85 (朱自清)
遞迴層[1] - /wiki/%E8%81%9E%E4%B8%80%E5%A4%9A (聞一多)
遞迴層[1] - /wiki/%E5%90%B4%E5%AE%93 (吳宓)
遞迴層[1] - /wiki/%E9%87%91%E5%B2%B3%E9%9C%96 (金岳霖)
遞迴層[1] - /wiki/%E5%8F%B6%E4%BC%81%E5%AD%99 (葉企孫)
遞迴層[1] - /wiki/%E5%90%B3%E6%9C%89%E8%A8%93 (吳有訓)
遞迴層[1] - /wiki/%E5%91%A8%E5%9F%B9%E6%BA%90 (周培源)
遞迴層[1] - /wiki/%E9%99%B3%E7%9C%81%E8%BA%AB (陳省身)
遞迴層[1] - /wiki/%E9%99%B3%E5%B2%B1%E5%AD%AB (陳岱孫)
遞迴層[1] - /wiki/%E7%8E%8B%E5%A3%AB%E5%80%AC (王士倬)
遞迴層[1] - /wiki/%E8%92%8B%E5%8D%97%E7%BF%94 (蔣南翔)
遞迴層[1] - /wiki/%E4%B8%80%E4%BA%8C%C2%B7%E4%B9%9D%E

KeyboardInterrupt: 